In [1]:
# Set up and Import Dependencies

import pandas as pd
import pymongo
import requests
import os
import time

from splinter import Browser
from bs4 import BeautifulSoup as bs
from webdriver_manager.chrome import ChromeDriverManager

# SCRAPING

In [2]:
# Setup splinter
# -------------------------------------------------
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Driver [/Users/ernestbondi/.wdm/drivers/chromedriver/mac64/91.0.4472.101/chromedriver] found in cache


In [3]:
# URL of page to be scraped

url = ('https://redplanetscience.com/')
browser.visit(url)

soup = bs(browser.html, 'html.parser')

In [4]:
#Collect News titles from url
news_title = soup.find('div', class_="content_title").get_text()
news_title

'NASA to Reveal Name of Its Next Mars Rover'

In [5]:
# Collect News title paragraphs from url
news_p = soup.find('div', class_="article_teaser_body").get_text()
news_p

"After a months-long contest among students to name NASA's newest Mars rover, the agency will reveal the winning name — and the winning student — this Thursday. "

# IMAGES

In [6]:
# URL of page to be scraped
url2 = 'https://spaceimages-mars.com/'
browser.visit(url2)

soup = bs(browser.html, 'html.parser')

In [7]:
relative_image_path = soup.find_all('a')[2]["href"]

featured_image_url = url2 + relative_image_path

featured_image_url

'https://spaceimages-mars.com/image/featured/mars1.jpg'

In [8]:
#FEATURED IMAGE
relative_image_path

'image/featured/mars1.jpg'

# FACTS

In [9]:
#URL
url = 'https://galaxyfacts-mars.com/'

In [10]:
tables = pd.read_html(url)
tables

[                         0                1                2
 0  Mars - Earth Comparison             Mars            Earth
 1                Diameter:         6,779 km        12,742 km
 2                    Mass:  6.39 × 10^23 kg  5.97 × 10^24 kg
 3                   Moons:                2                1
 4       Distance from Sun:   227,943,824 km   149,598,262 km
 5          Length of Year:   687 Earth days      365.24 days
 6             Temperature:     -87 to -5 °C      -88 to 58°C,
                       0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.39 × 10^23 kg (0.11 Earths)
 3                Moons:          2 ( Phobos & Deimos )
 4       Orbit Distance:       227,943,824 km (1.38 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                   -87 to -5 °C
 7         First Record:              2nd millennium BC

In [11]:
df = tables[0]
#df.head()
df.columns = ["Info","Mars","Earth"]
df

,Info,Mars,Earth
0,Mars - Earth Comparison,Mars,Earth
1,Diameter:,"6,779 km","12,742 km"
2,Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
3,Moons:,2,1
4,Distance from Sun:,"227,943,824 km","149,598,262 km"
5,Length of Year:,687 Earth days,365.24 days
6,Temperature:,-87 to -5 °C,-88 to 58°C


In [12]:
#REMOVE FIRST ROW
df = df.iloc[1:]
df

,Info,Mars,Earth
1,Diameter:,"6,779 km","12,742 km"
2,Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
3,Moons:,2,1
4,Distance from Sun:,"227,943,824 km","149,598,262 km"
5,Length of Year:,687 Earth days,365.24 days
6,Temperature:,-87 to -5 °C,-88 to 58°C


In [13]:
#TABLE TO HTML
html_table = df.to_html()
html_table

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Info</th>\n      <th>Mars</th>\n      <th>Earth</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>1</th>\n      <td>Diameter:</td>\n      <td>6,779 km</td>\n      <td>12,742 km</td>\n    </tr>\n    <tr>\n      <th>2</th>\n      <td>Mass:</td>\n      <td>6.39 × 10^23 kg</td>\n      <td>5.97 × 10^24 kg</td>\n    </tr>\n    <tr>\n      <th>3</th>\n      <td>Moons:</td>\n      <td>2</td>\n      <td>1</td>\n    </tr>\n    <tr>\n      <th>4</th>\n      <td>Distance from Sun:</td>\n      <td>227,943,824 km</td>\n      <td>149,598,262 km</td>\n    </tr>\n    <tr>\n      <th>5</th>\n      <td>Length of Year:</td>\n      <td>687 Earth days</td>\n      <td>365.24 days</td>\n    </tr>\n    <tr>\n      <th>6</th>\n      <td>Temperature:</td>\n      <td>-87 to -5 °C</td>\n      <td>-88 to 58°C</td>\n    </tr>\n  </tbody>\n</table>'

# MARS HEMISPHERE

In [14]:
# URL of page to be scraped
url3 = 'https://marshemispheres.com/'
browser.visit(url3)

soup = bs(browser.html, 'html.parser')

In [15]:
#GET A LIST OF ALL TITLES
title_list = []

titles = soup.find_all('h3')
titles

[<h3>Cerberus Hemisphere Enhanced</h3>,
 <h3>Schiaparelli Hemisphere Enhanced</h3>,
 <h3>Syrtis Major Hemisphere Enhanced</h3>,
 <h3>Valles Marineris Hemisphere Enhanced</h3>,
 <h3>Back</h3>]

In [16]:
for title in titles:
    title_list.append(title.text)

title_list

['Cerberus Hemisphere Enhanced',
 'Schiaparelli Hemisphere Enhanced',
 'Syrtis Major Hemisphere Enhanced',
 'Valles Marineris Hemisphere Enhanced',
 'Back']

In [17]:
# GET A LIST OF ALL URL'S
url_list = []

for title in title_list:
    url3 = 'https://marshemispheres.com/'
    browser.visit(url3)
    browser.click_link_by_partial_text(title)
    html = browser.html
    soup = bs(browser.html, 'html.parser')
    image_url = soup.find_all('li')[0].a["href"]
    dictionary = {"title": title,"image_url":url3+ image_url}
    url_list.append(dictionary)

/Users/ernestbondi/opt/anaconda3/envs/PythonData/lib/python3.8/site-packages/splinter/driver/webdriver/__init__.py:490: FutureWarning: browser.find_link_by_partial_text is deprecated. Use browser.links.find_by_partial_text instead.
  warnings.warn(


ElementNotInteractableException: Message: element not interactable
  (Session info: chrome=91.0.4472.101)


In [18]:
url_list

[{'title': 'Cerberus Hemisphere Enhanced',
  'image_url': 'https://marshemispheres.com/images/full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'image_url': 'https://marshemispheres.com/images/schiaparelli_enhanced-full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'image_url': 'https://marshemispheres.com/images/syrtis_major_enhanced-full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'image_url': 'https://marshemispheres.com/images/valles_marineris_enhanced-full.jpg'}]